## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [57]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

In [58]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://webapps1.chicago.gov/buildingrecords/")



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_mac64.zip
Driver has been saved in cache [/Users/luizfernandotoledo/.wdm/drivers/chromedriver/mac64/96.0.4664.45]
/var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/ipykernel_73647/3495999009.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [59]:
driver.find_element(By.ID, 'rbnAgreement1').click()
driver.find_element(By.ID, 'submit').click()

## Searching

Search for **400 E 41ST ST**.

In [60]:
driver.find_element(By.ID, 'fullAddress').send_keys('400 E 41ST ST')
driver.find_element(By.ID, 'submit').click()


## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [93]:
database = []
table = driver.find_element(By.TAG_NAME, 'tbody')
line = table.find_elements(By.TAG_NAME, 'tr')
for cell in line:
    data = {}
    element = cell.find_elements(By.TAG_NAME, 'td')
    data['permit'] = (element[0].text)
    data['date_issued'] = (element[1].text)
    data['description_of_work'] = (element[2].text)
    database.append(data)
    
database

[{'permit': '100845718',
  'date_issued': '10/11/2019',
  'description_of_work': 'ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2020.'},
 {'permit': '100778302',
  'date_issued': '08/17/2018',
  'description_of_work': "PERMIT EXPIRES ON 10/17/2018 Erection Starts: 8/24/2018, Erection Ends: 8/25/2018. TENT- 1 DAY EVENT 8/25/18 ERECT ONE (1) 40'X60' POLE TENT SETUP 8/24/18- REMOVE 8/25/18 ASSEMBLY **** CONDITIONAL APPROVAL **** SUBJECT TO FIELD INSPECTION **** AOR CERTIFICATE OF INSPECTION REQUIRED ****"},
 {'permit': '100721255',
  'date_issued': '08/24/2017',
  'description_of_work': 'PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: 8/25/2017, ERECTION ENDS: 8/26/2017. SELF CERTIFICATION. REINSTATEMENT OF 2015 PERMIT 100610771 FOR THE PAUL G STEWART CENTER EVENT. SAME 40X60 CANOPY. AAA RENTAL SYSTEM.** CONDITIONAL APPROVAL **SUBJECT TO FIELD INSPECTION * AOR CERTIFICATION OF INSPECTION REQUIRED.'},
 {'permit': '100693399',
  'date_issued': '03/03/2017',
  'description_of_work': 'INSTALLATI

In [99]:
df = pd.DataFrame(database)

In [100]:
df.to_csv('permits_400_e_41st_st.csv', index=False)

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [109]:
database2 = []
table = driver.find_element(By.XPATH, '/html/body/div/div[4]/div[10]/table/tbody')
line = table.find_elements(By.TAG_NAME, 'tr')
for cell in line:
    data = {}
    element = cell.find_elements(By.TAG_NAME, 'td')
    data['insp'] = (element[0].text)
    data['inspection_date'] = (element[1].text)
    data['status'] = (element[2].text)
    data['type_description'] = (element[3].text)
    database2.append(data)
    



In [110]:
df = pd.DataFrame(database2)

In [111]:
df.to_csv('inspections_400_e_41st_st.csv', index=False)

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.